# Magíster Data Science
## Evaluación Almacenamiento y Captura de datos
### Profesor : Claudio Aracena
--- 

#### Instrucciones : 

En grupos de máximo tres personas deberán resolver los dos problemas descritos abajo.
Deben entregar el notebook con el código para resolver cada problema. 
Agregue comentarios al código para entender el razonamiento utilizado.

Plazo de Entrega : 01-Marzo-2020  23:59:59

#### Integrantes : 
*Roy Barrera; Jorge Cotapos; José Pérez*

---

#### 1) Realice un proceso de web scraping para el sitio https://www.portalinmobiliario.com/.
Para ello realice las siguientes tareas:

a. Utilizar Selenium para emular un navegador y cargar la página requerida (1
punto).

In [1]:
# Carga las librería requeridas

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import time

In [2]:
# Abre el Navegador y Carga la página solicitada

driver = webdriver.Chrome(ChromeDriverManager().install())
portal='http://www.portalinmobiliario.com/'
driver.get(portal)



Looking for [chromedriver 80.0.3987.106 linux64] driver in cache 
File found in cache by path [/home/pepeperez/.wdm/drivers/chromedriver/80.0.3987.106/linux64/chromedriver]


b. Buscar arriendo de departamentos en la comuna de Santiago utilizando los
campos de inputs del buscador de la página (1 punto).

In [3]:
# Selecciona el elemento searchbox-realstate

caja = driver.find_element_by_class_name('searchbox')


In [4]:
# Selecciona la operación de Arriendo

operacion = caja.find_element_by_id('operations-dropdown')

oper = operacion.find_element_by_class_name('andes-dropdown')
oper.click() # abre la lista 

oper_arriendo = oper.find_element_by_css_selector("[data-id='arriendo']")
oper_arriendo.click() # selecciona la operacion "arriendo"


In [5]:
# Seleccina el tipo (o categoría) Departamento

categoria = caja.find_element_by_id('categories-dropdown')

categ = categoria.find_element_by_class_name('andes-dropdown')
categ.click() # abre la lista

categ_depto = categ.find_element_by_css_selector("[data-id='arriendo_departamento']")
categ_depto.click() # selecciona la categoría "departamento"


In [6]:
# Ingresa Ubicación (Location) del arriendo buscado

ubic = caja.find_element_by_class_name('searchbox-autocomplete__field')

ubic.send_keys('Santiago, RM(Metropolitana)') # ingresa el texto a la caja de diálogo

time.sleep(1) # se necesita un momento para que se active la lista desplegable

# Selecciona la ubicación 

ubic_combo = caja.find_element_by_class_name('searchbox-autocomplete-dropdown')

ubic_stgo = ubic_combo.find_element_by_css_selector('[data-id="TUxDQ1NBTjk4M2M"]')
ubic_stgo.click() # selecciona la ubicación "Santiago, RM (Metropolitana)"


---

Hasta aquí hemos seleccionado :
- Operación : Arriendo
- Categoría : Departamento
- Ubicación : Santiago, RM (Metropolitana)

Cumpliendo así los parámetros de la tarea.

Si se hubiera pedido, además, seleccionar solamente los nuevos proyectos, sería necesario aquí escoger el checkbox "solo nuevos" y hacer __*Click*__  en él, antes de continuar.

---

In [7]:
# Click en el botón buscar, para cambiar de página

buscar = driver.find_element_by_class_name('searchbox-button__content')     
buscar.click()


c. Extraer los primeros 100 departamentos con su precio, superficie, cantidad
de dormitorios, cantidad de baños y dirección (1 punto).

In [8]:
# define la función xpath_text

def xpath_text(xpath):
    try: return driver.find_element_by_xpath(xpath).text
    except: return ''


__Nota :__ 
- Se nos ha pedido los primeros 100 resultados, pero hemos descubierto que se muestran solo 50 resultados por página, lo cual implica que debemos navegar para avanzar a las páginas siguientes para obtener el resto de los resultados esperados

In [9]:
# Preparación para el rescate

# define las listas para guardar los datos que vamos a rescatar de la página

precios = []
monedas = []
caracteristicas = []
direcciones = []

# define las partes fijas de la ruta del rescate

prefijo = '/html/body/main/div/div/section/ol/li['
sufijo  = ']/div/div[2]/div/'

# define las variables según los parámetros de la tarea 

resultados_x_pagina = 50
resultados_deseados = 100


In [10]:
# Rescata los Departamentos en Arriendo en Santiago, RM (Metropolitana)

while len(caracteristicas) < resultados_deseados:
    
    # rescata los resultados posibles de la página actual  (a la que llegamos navegando)
    
    for i in range(resultados_x_pagina) :
        precios.append(xpath_text        ( prefijo + str(i+1) + sufijo + 'a/div/div/span[2]'))
        monedas.append(xpath_text        ( prefijo + str(i+1) + sufijo + 'a/div/div/span[1]'))
        caracteristicas.append(xpath_text( prefijo + str(i+1) + sufijo + 'div/a[1]/div'))
        direcciones.append(xpath_text    ( prefijo + str(i+1) + sufijo + 'div/a[2]/span'))

    # Avanza (Navega) a la siguiente página 

    paginacion = driver.find_element_by_class_name('pagination__container')
    boton_sgte = paginacion.find_element_by_class_name('andes-pagination__button--next')
    boton_sgte.click()
    
    
# validación al final
len(caracteristicas)

100

In [11]:
# se cierra el driver al terminar de usarlo

driver.close()

d. Crear un dataframe con la información extraída (1 punto).

In [12]:
# Carga las librería requeridas

import pandas as pd
import numpy as np

In [13]:
# Define las listas necesarias para "abrir" las características

metros = []
dormitorios = [] 
baños = []
precioentero =[]

for i in range(len(caracteristicas)) : 
    texto = caracteristicas[i].split('|')
    metros.append(texto[0])
    dormitorios.append(texto[1])  
    baños.append(texto[2])



In [14]:
# crea el dataframe requerido

df = pd.DataFrame({
    'Moneda': monedas,
    'Precio': precios,
    'metraje': metros,    
    'dormitorios': dormitorios,    
    'baños': baños,
    'Dirección': direcciones
})

# validación
df

,Moneda,Precio,metraje,dormitorios,baños,Dirección
0,UF,"10,10",25 - 53 m² útiles,1 - 2 dormitorios,1 - 2 baños,"Ejército 71, Santiago"
1,UF,11,25 - 45 m² útiles,1 - 2 dormitorios,1 - 2 baños,"Santa Rosa 237, Santiago"
2,UF,"10,88",26 - 44 m² útiles,1 - 2 dormitorios,1 - 2 baños,"Santa Isabel 55, Santiago"
3,$,255.000,40 m² útiles,1 dormitorio,1 baño,Barrio Yungay: Arriendo Departamento Tipo Loft...
4,$,230.000,23 m² útiles,1 dormitorio,1 baño,"Cienfuegos, Santiago - Departamento 78, Santiago"
...,...,...,...,...,...,...
95,$,280.000,45 m² útiles,1 dormitorio,1 baño,"Huérfanos 1400, Santiago"
96,$,320.000,33 m² útiles,1 dormitorio,1 baño,"Metro Santa Lucia, Santiago"
97,$,390.000,60 m² útiles,2 dormitorios,1 baño,"Portugal 564 / Metro Sta. Isabel, Santiago"
98,$,330.000,30 m² útiles,1 dormitorio,1 baño,"Enrique Mac Iver 524, Santiago, Santiago"


e. Crear una base de datos con la información del dataframe (1 punto).

In [15]:
# importa las librerias necesarias

import sqlite3

#!pip install sqlalchemy
from sqlalchemy import create_engine


In [16]:
# crea una base de datos para la tarea y establece una conexión con ella

sql_engine = create_engine('sqlite:///tarea1.db')
conn = sql_engine.raw_connection()


In [17]:
# Elimina la tabla 'portal', si es que existe previamente

cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS portal")

In [18]:
# a partir del dataframe, crea la tabla 'portal' en la base de datos

df.to_sql('portal', conn, index=False)

f. Realizar una consulta SQL para saber el precio promedio de arriendo por
metro cuadrado (1 punto).

In [19]:
# Consulta preliminar para ver los datos en la tabla.  Valores en Pesos

pd.read_sql('SELECT moneda, '
                   'precio, '
                   'substr(metraje,1,instr(metraje," ")-1) as metros, '
                   'precio/substr(metraje,1,instr(metraje," ")-1) as valor_metro, '
                   'precio * 1000 as precio_real, '
                   '( precio * 1000 )/substr(metraje,1,instr(metraje," ")-1) as valor_real '
            'FROM portal '
            'WHERE moneda = "$" '
            'ORDER BY cast(substr(metraje,1,instr(metraje," ")-1) as decimal) desc'
            , conn)


,Moneda,Precio,metros,valor_metro,precio_real,valor_real
0,$,875.000,135,6.481481,875000.0,6481.481481
1,$,690.000,120,5.750000,690000.0,5750.000000
2,$,660.000,120,5.500000,660000.0,5500.000000
3,$,910.000,116,7.844828,910000.0,7844.827586
4,$,600.000,80,7.500000,600000.0,7500.000000
...,...,...,...,...,...,...
86,$,270.000,29,9.310345,270000.0,9310.344828
87,$,280.000,27,10.370370,280000.0,10370.370370
88,$,350.000,26,13.461538,350000.0,13461.538462
89,$,280.000,25,11.200000,280000.0,11200.000000


In [20]:
# Determina el valor promedio del metro cuadrado, en PESOS

pd.read_sql('SELECT round(avg(valor)) as valor_promedio_metro_cuadrado '
            'FROM ('
                   'SELECT '
                         '(precio *1000)/substr(metraje,1,instr(metraje," ")-1) as valor '
                   'FROM portal '
                   'WHERE moneda = "$" '
                 ') ', conn)


,valor_promedio_metro_cuadrado
0,8527.0


__NOTA :__

Para el caso de los proyectos, el valor se pone en UF (valor desde) y el metraje se expresa en 2 valores (desde-hasta) por lo tanto, si nos quedamos con el primer metraje, en ambos casos estaremos con el valor mínimo (desde)

In [21]:
# Consulta preliminar para ver los datos en la tabla.  Valores en UFs

pd.read_sql('SELECT moneda, '
                   'precio, '
                   'metraje,'
                   'cast(substr(metraje,1,instr(metraje," ")-1) as decimal) as mt_desde, '
                   'substr(metraje,instr(metraje,"-")+1, instr(metraje," ") ) as mt_hasta, '
                   'round(cast(replace(precio,",",".") as decimal),2) as precio_real, '
                   '(round(cast(replace(precio,",",".") as decimal),2)/'
                   'cast(substr(metraje,1,instr(metraje," ")-1) as decimal)) as valor_mt2 '
            'FROM portal '
            'WHERE moneda != "$" '
            'ORDER BY cast(substr(metraje,1,instr(metraje," ")-1) as decimal) desc'
            , conn)


,Moneda,Precio,metraje,mt_desde,mt_hasta,precio_real,valor_mt2
0,UF,"16,95",96 m² útiles,96,96,16.95,0.176562
1,UF,16,62 m² útiles,62,62,16.00,0.258065
2,UF,25,56 m² útiles,56,56,25.00,0.446429
3,UF,13,54 m² útiles,54,54,13.00,0.240741
4,UF,"12,70",50 m² útiles,50,50,12.70,0.254000
5,UF,"11,42",31 m² útiles,31,31,11.42,0.368387
6,UF,"10,88",26 - 44 m² útiles,26,44,10.88,0.418462
7,UF,"10,10",25 - 53 m² útiles,25,53,10.10,0.404000
8,UF,11,25 - 45 m² útiles,25,45,11.00,0.440000


In [22]:
# Determina el valor promedio del metro cuadrado, en UFs

pd.read_sql('SELECT avg(valor) as valor_promedio_metro_cuadrado '
            'FROM ('
                   'SELECT '
                         '(round(cast(replace(precio,",",".") as decimal),2)/cast(substr(metraje,1,instr(metraje," ")-1) as decimal)) as valor '
                   'FROM portal '
                   'WHERE moneda != "$" '
                 ') ', conn)


,valor_promedio_metro_cuadrado
0,0.334072


__Nota :__
- ¿Cual es el valor actual de la UF?


__BONUS (para el "punto extra")__


In [23]:
# Importa Librerías necesarias.

#!pip install beautifulsoup4
#!pip install requests

from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests


In [24]:
# Para rescatar el valor Actual de la UF en Chile, busca la página más simple

html = urlopen("https://valoruf.cl")
res = BeautifulSoup(html.read(),"html5lib")
print(res.title.text)


⭐ Valor UF | Valor oficial de la UF hoy | Convertir unidad de fomento a pesos


In [25]:
# Rescata el valor actual (vpr : valor presente)

req = requests.get("https://valoruf.cl" )
res = BeautifulSoup(req.text,"html.parser")
valor_uf = res.find('span', {'class': 'vpr'}).text
valor_uf


'$ 28.469,54'

In [26]:
# convierte el valor en una lista de numeros (con un solo elemento)

uf = [] 
uf.append(valor_uf.replace("$ ","").replace(".","").replace(",","."))
uf


['28469.54']

In [27]:
# Elimina la tabla 'uf', si es que existe previamente

cursor.execute("DROP TABLE IF EXISTS uf")


In [28]:
# Carga el valor actual a la base de datos

df2 = pd.DataFrame({'Valor': uf })
df2.to_sql('UF', conn, index=False)
pd.read_sql("SELECT valor FROM uf", conn)


,Valor
0,28469.54


In [29]:
# Repite el calculo del valor promedio, agregando la conversión a Pesos

pd.read_sql('SELECT avg(valor) as valor_promedio_mt2_UF ,'
                   'round (avg(valor) * ( SELECT valor FROM uf )) as  valor_equivalente_PESOS '
            'FROM ('
                   'SELECT '
                         '(round(cast(replace(precio,",",".") as decimal),2)/cast(substr(metraje,1,instr(metraje," ")-1) as decimal)) as valor '
                   'FROM portal '
                   'WHERE moneda != "$" '
                 ') ', conn)



,valor_promedio_mt2_UF,valor_equivalente_PESOS
0,0.334072,9511.0


In [30]:
# cierra la conección con la base de datos

conn.close()



---

**Agradecimientos :**
- Damos las gracias a nuestro compañero **_Sergio Arancibia_** por su valiosa colaboración al trabajo de este equipo.

---



#### 2) Utilizando la API de Twitter, realice las siguientes tareas:

a. Extraer los últimos 1.000 tweets de tu usuario preferido que escriba en
español (1 punto)

In [31]:
# Es importada la librería tweepy, para el uso de la api de twitter.
import tweepy
# Es importada la librería TweepError, para manejar error cuando el id al ser accesado da error.
from tweepy.error import TweepError

# Se define la función trae_accesos_twitter(), para obtener los string de autenticación para los api de twitter.
# Por defecto en el archivo Acceso_twitter.txt, se encuentran los string de autenticación. El archivo Acceso_twitter.txt,
# se entregará con esta tarea, y debe quedar en la misma carpeta en la que abra este Jupyter Notebook.
def trae_accesos_twitter(archivo='Acceso_twitter.txt'):
    with open(archivo) as f:
        lineas = f.readlines()
    consumer_key        = lineas[0][:len(lineas[0])-1]
    consumer_secret     = lineas[1][:len(lineas[1])-1]
    access_token        = lineas[2][:len(lineas[2])-1]
    access_token_secret = lineas[3][:len(lineas[3])-1]
    
    return consumer_key, consumer_secret, access_token, access_token_secret

# Se obtienen los string de autenticación, llamando a la función trae_accesos_twitter()
consumer_key, consumer_secret, access_token, access_token_secret = trae_accesos_twitter()

# Se realizan las autenticaciones 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Se instancia la api tweepy.API()
api = tweepy.API(auth)


In [32]:
# Es definida la función extrae_tweet(api,id,n_tweet), de parámetros api, id, n_twteet, dónde:
# api corresponde a la api tweepy.API() ya instanciada
# id corresponde al id del twiter desde el que se quieren extraer los tweet, por defecto '@@UDD_cl'
# n_tweet corresponde a la cantidad de tweet a extraer, por defecto 1000
# Retorna list_tweet[] que es una lista que contiene los textos de cada tweet junto con
# su fecha de creación

def extrae_tweet(api=api, id='@UDD_cl', n_tweet=1000):
    list_tweet=[]
    for tweet in tweepy.Cursor(api.user_timeline,id=id).items(n_tweet):
        list_tweet.append((tweet.created_at,tweet.text))
    return list_tweet

# Son extraidos los últimos 1.000 tweet de @UDD_cl y se dejan en UDD_tweet
UDD_tweet = extrae_tweet(api, '@UDD_cl',1000)

# Se comprueba que los 1.000 tweet fueron obtenidos
len(UDD_tweet), UDD_tweet

(1000,
 [(datetime.datetime(2020, 2, 27, 20, 17, 38),
   'RT @UDD_FM: Dra. Ximena Aguilera: "La probabilidad de que COVID-19 llegue a Chile es cada vez más alta"\n\nDirectora del Centro de Epidemiolo…'),
  (datetime.datetime(2020, 2, 27, 19, 35, 2),
   '@yeyitamunozaray No trabaja con nosotros, saludos'),
  (datetime.datetime(2020, 2, 27, 19, 34, 23),
   '@Pa__tty Hola, Maximiliano Pérez-Iñigo no trabaja con nosotros. Saludos'),
  (datetime.datetime(2020, 2, 27, 19, 33, 56),
   '@Prensadespiert1 @illapu @ValdebenitoNata @AnaMariaGazmuri @amarolabra @Su_Hidalgo @danieljadue Hola, Maximiliano P… https://t.co/BHcZmVc4ei'),
  (datetime.datetime(2020, 2, 27, 19, 33, 35),
   '@nickfps @Prensadespiert1 @illapu @ValdebenitoNata @AnaMariaGazmuri @amarolabra @Su_Hidalgo @danieljadue No trabaja con nosotros. Saludos'),
  (datetime.datetime(2020, 2, 27, 19, 33, 11),
   '@neuro_p_salinas Maximiliano Pérez-Iñigo no trabaja con nosotros. Saludos'),
  (datetime.datetime(2020, 2, 27, 13, 19, 10),
   'R

b. Hacer un análisis de frecuencia de palabras (100 palabras más frecuentes),
habiendo removido primero stopwords y usando tokenización en español (2
puntos).

In [33]:
# Se importan las librerías requeridas para hacer el análisis de frecuencias de palabaras.
# Se importa la librería stopwords para remover los stopwords según lo indicado.
from nltk.corpus import stopwords
# Se importa la librería ToktokTokenizer para realizar la tokenización en español solicitada.
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import word_tokenize, regexp_tokenize
# Se importa la libreria usada para contar las palabras más frecuentes.
from collections import Counter

In [34]:
# Se define la función tweet_to_string() que guarda el listado de tweet extraidos en un string. Esto se hace para el
# tratamiento posterior.
def tweets_to_string(l_tweets):
    s=''
    for element in l_tweets:
        s += element[1]+' '
    return s

# Son guardados en el string s los últimos 1.000 tweet extraidos de @UDD_cl.
s=''
s = tweets_to_string(UDD_tweet)
# Es mostrado el string s
s

'RT @UDD_FM: Dra. Ximena Aguilera: "La probabilidad de que COVID-19 llegue a Chile es cada vez más alta"\n\nDirectora del Centro de Epidemiolo… @yeyitamunozaray No trabaja con nosotros, saludos @Pa__tty Hola, Maximiliano Pérez-Iñigo no trabaja con nosotros. Saludos @Prensadespiert1 @illapu @ValdebenitoNata @AnaMariaGazmuri @amarolabra @Su_Hidalgo @danieljadue Hola, Maximiliano P… https://t.co/BHcZmVc4ei @nickfps @Prensadespiert1 @illapu @ValdebenitoNata @AnaMariaGazmuri @amarolabra @Su_Hidalgo @danieljadue No trabaja con nosotros. Saludos @neuro_p_salinas Maximiliano Pérez-Iñigo no trabaja con nosotros. Saludos RT @negociosUDD: 🔴 OPINIÓN| Compartimos columna de nuestro director del #CIES, @hernancheyre “"Base cero" ajustado” en @EYN_ELMERCURIO http… RT @porlotoole: Desde @iCubo_udd @UDD_cl feliz de estar trabajando con los profesores de colegios Dunalastair para enseñarles nuevas formas… RT @elliberocl: @MaidaOlea @AgriculturaFM Conversamos con Federico Valdés (@FedericoValdesL), Recto

In [35]:
# Se define la función fn_stop_words(s,l_sw), que amplia el listado de stop_words usando el listado l_sw
def fn_stop_words(s,l_sw):
    stop_words  = []
    stop_words  = list(stopwords.words('spanish')) 
## agregamos más stopwords
    stop_words += l_sw
    return len(stop_words), stop_words

l_sw = [',', '.', ':', ';','…','”','"','!','¡','...','%',"'",')','(','[',']','¿','?','🙌🏽','💪💪💪','👏👏👏👏','🔺','🔴','🔻']

len_stop_words, stop_words = fn_stop_words(s,l_sw)

len_stop_words, stop_words

(337,
 ['de',
  'la',
  'que',
  'el',
  'en',
  'y',
  'a',
  'los',
  'del',
  'se',
  'las',
  'por',
  'un',
  'para',
  'con',
  'no',
  'una',
  'su',
  'al',
  'lo',
  'como',
  'más',
  'pero',
  'sus',
  'le',
  'ya',
  'o',
  'este',
  'sí',
  'porque',
  'esta',
  'entre',
  'cuando',
  'muy',
  'sin',
  'sobre',
  'también',
  'me',
  'hasta',
  'hay',
  'donde',
  'quien',
  'desde',
  'todo',
  'nos',
  'durante',
  'todos',
  'uno',
  'les',
  'ni',
  'contra',
  'otros',
  'ese',
  'eso',
  'ante',
  'ellos',
  'e',
  'esto',
  'mí',
  'antes',
  'algunos',
  'qué',
  'unos',
  'yo',
  'otro',
  'otras',
  'otra',
  'él',
  'tanto',
  'esa',
  'estos',
  'mucho',
  'quienes',
  'nada',
  'muchos',
  'cual',
  'poco',
  'ella',
  'estar',
  'estas',
  'algunas',
  'algo',
  'nosotros',
  'mi',
  'mis',
  'tú',
  'te',
  'ti',
  'tu',
  'tus',
  'ellas',
  'nosotras',
  'vosotros',
  'vosotras',
  'os',
  'mío',
  'mía',
  'míos',
  'mías',
  'tuyo',
  'tuya',
  'tuyos',


In [36]:
# Se define la función tokenizador(toktok,s), donde toktok es el toquenizador a usar y s es el string a ser toquenizado.
def tokenizador(toktok, s):
    toktok      = toktok()
    word_tokens = toktok.tokenize(s.lower())
    return len(word_tokens), word_tokens

# Se usa ToktokTokenizer() que es mejor para español, pero contiene caracteres como punto, #, comas, etc.
ln_wt, l_wt = tokenizador(ToktokTokenizer,s)
ln_wt, l_wt

(22017,
 ['rt',
  '@udd_fm',
  ':',
  'dra.',
  'ximena',
  'aguilera',
  ':',
  '"',
  'la',
  'probabilidad',
  'de',
  'que',
  'covid-19',
  'llegue',
  'a',
  'chile',
  'es',
  'cada',
  'vez',
  'más',
  'alta',
  '"',
  'directora',
  'del',
  'centro',
  'de',
  'epidemiolo',
  '…',
  '@yeyitamunozaray',
  'no',
  'trabaja',
  'con',
  'nosotros',
  ',',
  'saludos',
  '@pa__tty',
  'hola',
  ',',
  'maximiliano',
  'pérez-iñigo',
  'no',
  'trabaja',
  'con',
  'nosotros.',
  'saludos',
  '@prensadespiert1',
  '@illapu',
  '@valdebenitonata',
  '@anamariagazmuri',
  '@amarolabra',
  '@su_hidalgo',
  '@danieljadue',
  'hola',
  ',',
  'maximiliano',
  'p',
  '…',
  'https://t.co/bhczmvc4ei',
  '@nickfps',
  '@prensadespiert1',
  '@illapu',
  '@valdebenitonata',
  '@anamariagazmuri',
  '@amarolabra',
  '@su_hidalgo',
  '@danieljadue',
  'no',
  'trabaja',
  'con',
  'nosotros.',
  'saludos',
  '@neuro_p_salinas',
  'maximiliano',
  'pérez-iñigo',
  'no',
  'trabaja',
  'con',
 

In [37]:
# Se define la función filtra_palabras(l_w,l_sw), en donde l_w es un listado de palabras a ser filtrado por las stop_words
# que están en l_sw
def filtra_palabras(l_w,l_sw):
    filtered_words =[w for w in l_w if not w in l_sw]
    return len(filtered_words), filtered_words

# Se filtran las palabras stop_words del listado de palabaras toquenizadas l_wt
ln_fw, l_fw = filtra_palabras(l_wt,l_sw)
# Se muestra el listado de palabras ya filtrado 
ln_fw, l_fw

(18944,
 ['rt',
  '@udd_fm',
  'dra.',
  'ximena',
  'aguilera',
  'la',
  'probabilidad',
  'de',
  'que',
  'covid-19',
  'llegue',
  'a',
  'chile',
  'es',
  'cada',
  'vez',
  'más',
  'alta',
  'directora',
  'del',
  'centro',
  'de',
  'epidemiolo',
  '@yeyitamunozaray',
  'no',
  'trabaja',
  'con',
  'nosotros',
  'saludos',
  '@pa__tty',
  'hola',
  'maximiliano',
  'pérez-iñigo',
  'no',
  'trabaja',
  'con',
  'nosotros.',
  'saludos',
  '@prensadespiert1',
  '@illapu',
  '@valdebenitonata',
  '@anamariagazmuri',
  '@amarolabra',
  '@su_hidalgo',
  '@danieljadue',
  'hola',
  'maximiliano',
  'p',
  'https://t.co/bhczmvc4ei',
  '@nickfps',
  '@prensadespiert1',
  '@illapu',
  '@valdebenitonata',
  '@anamariagazmuri',
  '@amarolabra',
  '@su_hidalgo',
  '@danieljadue',
  'no',
  'trabaja',
  'con',
  'nosotros.',
  'saludos',
  '@neuro_p_salinas',
  'maximiliano',
  'pérez-iñigo',
  'no',
  'trabaja',
  'con',
  'nosotros.',
  'saludos',
  'rt',
  '@negociosudd',
  'opinión

In [38]:
# Se define función palabras_frecuentes(l_w,n_w), que toma de entrada l_w, un listado de palabras y n_w que es la 
# cantidad de palabras mas fercuentes a considerar.
def palabras_frecuentes(l_w,n_w):
    counter = Counter(l_w)
    return counter.most_common(n_w)
# Se llama a la función palabras_frecuentes(), para conoces las 100 palabras más frecuentes. 
l_pf = palabras_frecuentes(l_fw,100)
l_pf

[('de', 1224),
 ('la', 733),
 ('en', 593),
 ('rt', 585),
 ('el', 511),
 ('y', 350),
 ('&#124;', 347),
 ('del', 345),
 ('a', 307),
 ('que', 232),
 ('los', 185),
 ('las', 150),
 ('con', 137),
 ('@udd_cl', 128),
 ('para', 127),
 ('se', 127),
 ('por', 113),
 ('sobre', 112),
 ('un', 105),
 ('@negociosudd', 101),
 ('es', 98),
 ('hoy', 98),
 ('nuestro', 87),
 ('no', 78),
 ('director', 78),
 ('una', 75),
 ('@udd_fm', 64),
 ('al', 62),
 ('chile', 55),
 ('#prensaudd', 54),
 ('e', 54),
 ('@investigaudd', 52),
 ('udd', 52),
 ('seminario', 51),
 ('facultad', 48),
 ('centro', 43),
 ('más', 41),
 ('directora', 41),
 ('@cics_udd', 40),
 ('alumnos', 40),
 ('rector', 38),
 ('innovación', 38),
 ('lo', 37),
 ('valdés', 35),
 ('su', 35),
 ('junto', 35),
 ('federico', 33),
 ('#negociosudd', 33),
 ('@pallards', 32),
 ('estudiantes', 32),
 ('nuestra', 30),
 ('fue', 30),
 ('charla', 30),
 ('programa', 29),
 ('@arquitecturaudd', 29),
 ('este', 29),
 ('te', 28),
 ('social', 28),
 ('instituto', 28),
 ('donde', 27

c. Extraer 100 usuarios que son seguidos por su usuario de preferencia y
extraer los últimos 10 tweets de cada uno de esos usuarios (1 punto).

In [39]:
# Se define la función trae_usuarios_seguidos(id,cantidad), donde id es el twitter que se quiere saber que seguidores tiene y 
# cantidad, es la cantidad de seguidores a considerar.
# Se valida que los twitter seguidos por @UDD_cl no den error al traer sus tweet. Este error surgio durante las pruebas, hay 
# twitter seguidos po @UDD_cl que dan error 401 al ser intentar extraer sus tweet.
def trae_usuarios_seguidos(id='@UDD_cl',cantidad=100):
    ids_rigth=[]
    ids_fail=[]
    i=0
    usuarios_seguidos_ids = api.friends_ids(id)
    for id in usuarios_seguidos_ids:
        if i > cantidad:
            ids_rigth.append(id)
        else:
            try:
                extrae_tweet(api, id, 1)
                ids_rigth.append(id)
            except TweepError:
                ids_fail.append(id)
        i += 1

                
    return ids_rigth[0:cantidad]

# Se obtiene 100 seguidores de @UDD_cl
l_seguidos = trae_usuarios_seguidos('@UDD_cl',100)
len(l_seguidos), l_seguidos

(100,
 [13592012,
  215464834,
  1133772716341907462,
  50790808,
  338689270,
  204441838,
  871358839,
  1449927013,
  244334937,
  1629183134,
  44136211,
  53763883,
  221313572,
  2712291697,
  939106239627382788,
  396796970,
  1576383836,
  39991189,
  1116059487578853376,
  1131674157295513603,
  16392202,
  245388810,
  1467717914,
  2279120598,
  1158375356434407424,
  633098103,
  30174232,
  78429806,
  964260014813794311,
  2986223787,
  104817370,
  702881306066198528,
  63686819,
  390146520,
  984137106753810435,
  1034460436027125764,
  33989170,
  796471599839014912,
  44980318,
  101614836,
  730771151354535936,
  846089316,
  1024286742222917632,
  203186089,
  1083243070588624896,
  104959191,
  18121736,
  64449668,
  335436884,
  238162336,
  2456754781,
  60685242,
  69927300,
  30871522,
  123859769,
  3514533617,
  325612031,
  66259804,
  860595897683640320,
  122131132,
  14159148,
  43968262,
  266751132,
  165290523,
  1002344005,
  3458508495,
  402466434

In [40]:
# Se traen los 10 últimos tweet de 100 twitter seguidos por @UDD_cl y se guardan en l_seguidos_tweet 
l_seguidos_tweet = []
ids_fail=[]
for seguidos in l_seguidos:
    try:
        l_seguidos_tweet.append(extrae_tweet(api, seguidos,10))
    except TweepError:
        ids_fail.append(seguidos)
        
len(l_seguidos_tweet),l_seguidos_tweet

(100,
 [[(datetime.datetime(2020, 3, 1, 20, 50, 47),
    '4 Things Every Business Leader Should Know About Artificial Intelligence and Automation https://t.co/DYkkXXkRqY'),
   (datetime.datetime(2020, 3, 1, 17, 43, 42),
    'RT @EduardoBitran: Recomiendo esta entrevista a Acemoglu, Economista MIT. Autor del libro Porque fracasan los países? A favor de nueva Cons…'),
   (datetime.datetime(2020, 2, 29, 15, 15, 46),
    'Big Corporations Are Rushing Into Venture Capital. That May Not Be a Good Thing. https://t.co/dffGpGIhE6'),
   (datetime.datetime(2020, 2, 27, 19, 1, 20),
    'Pathways to Growth: Innovation is essential, but not enough https://t.co/oJoWDrC58o'),
   (datetime.datetime(2020, 2, 26, 20, 46, 31),
    '@Forbes https://t.co/k7ZWTOVdbZ'),
   (datetime.datetime(2020, 2, 26, 19, 58, 47),
    'Hiring A Chief Innovation Officer Is A Bad Idea via @forbes https://t.co/i8wTVDIS5b'),
   (datetime.datetime(2020, 2, 19, 22, 2, 4),
    'RT @UDDVentures: Nuestro partner InterSystems está l

In [41]:
# Se realiza una primera interpretación del ejercicio 2.d. en donde el análisis de frecuencia de palabras se hace en forma
# separada por cada unos los twitter seguidos por @UDD_cl. Es decir se hace el análisis de frecuencia de palabra para cada 
# uno de los 100 twitter seguidos por @UDD_cl seleccionados.
# Son llevados a strins los 10 tweet de cada uno de los 100 twitter seguidos por @UDD_cl
l_s=[]
for tweet in l_seguidos_tweet:
    l_s.append(tweets_to_string(tweet))
# Es mostrado el l_s
len(l_s), l_s

(100,
 ['4 Things Every Business Leader Should Know About Artificial Intelligence and Automation https://t.co/DYkkXXkRqY RT @EduardoBitran: Recomiendo esta entrevista a Acemoglu, Economista MIT. Autor del libro Porque fracasan los países? A favor de nueva Cons… Big Corporations Are Rushing Into Venture Capital. That May Not Be a Good Thing. https://t.co/dffGpGIhE6 Pathways to Growth: Innovation is essential, but not enough https://t.co/oJoWDrC58o @Forbes https://t.co/k7ZWTOVdbZ Hiring A Chief Innovation Officer Is A Bad Idea via @forbes https://t.co/i8wTVDIS5b RT @UDDVentures: Nuestro partner InterSystems está lanzando una funcionalidad global para su sistema unificado de información de salud Trak… RT @UDDVentures: "Innovación en la sociedad" Columna de Angel Morales sobre la importancia del aporte de la innovación para generar nuevos… RT @UDDVentures: Toma la decisión correcta para lograr hacer crecer tu negocio. Asegura tu cupo en el programa UDD Ventures Transforma. Esc… @RobertoKPa

In [42]:
# Se toquenizan los 10 últimos tweet de los 100 twitter seguidos por @UDD_cl
l_ls_wt=[]
for s in l_s:
    l_ls_wt.append(tokenizador(ToktokTokenizer,s))
len(l_ls_wt),l_ls_wt 

(100,
 [(158,
   ['4',
    'things',
    'every',
    'business',
    'leader',
    'should',
    'know',
    'about',
    'artificial',
    'intelligence',
    'and',
    'automation',
    'https://t.co/dykkxxkrqy',
    'rt',
    '@eduardobitran',
    ':',
    'recomiendo',
    'esta',
    'entrevista',
    'a',
    'acemoglu',
    ',',
    'economista',
    'mit.',
    'autor',
    'del',
    'libro',
    'porque',
    'fracasan',
    'los',
    'países',
    '?',
    'a',
    'favor',
    'de',
    'nueva',
    'cons',
    '…',
    'big',
    'corporations',
    'are',
    'rushing',
    'into',
    'venture',
    'capital.',
    'that',
    'may',
    'not',
    'be',
    'a',
    'good',
    'thing.',
    'https://t.co/dffgpgihe6',
    'pathways',
    'to',
    'growth',
    ':',
    'innovation',
    'is',
    'essential',
    ',',
    'but',
    'not',
    'enough',
    'https://t.co/ojowdrc58o',
    '@forbes',
    'https://t.co/k7zwtovdbz',
    'hiring',
    'a',
    'chief',
 

In [43]:
# Se filtran las palabras stop_words del listado de palabaras toquenizadas los 10 últimos tweet de los 100 twitter seguidos
# por @UDD_cl
l_ls_fw=[]
for i in range(len(l_ls_wt)):
    l_ls_fw.append(filtra_palabras(l_ls_wt[i][1],l_sw))
# Se muestra el listado de palabras ya filtrado 
len(l_ls_fw), l_ls_fw

(100,
 [(144,
   ['4',
    'things',
    'every',
    'business',
    'leader',
    'should',
    'know',
    'about',
    'artificial',
    'intelligence',
    'and',
    'automation',
    'https://t.co/dykkxxkrqy',
    'rt',
    '@eduardobitran',
    'recomiendo',
    'esta',
    'entrevista',
    'a',
    'acemoglu',
    'economista',
    'mit.',
    'autor',
    'del',
    'libro',
    'porque',
    'fracasan',
    'los',
    'países',
    'a',
    'favor',
    'de',
    'nueva',
    'cons',
    'big',
    'corporations',
    'are',
    'rushing',
    'into',
    'venture',
    'capital.',
    'that',
    'may',
    'not',
    'be',
    'a',
    'good',
    'thing.',
    'https://t.co/dffgpgihe6',
    'pathways',
    'to',
    'growth',
    'innovation',
    'is',
    'essential',
    'but',
    'not',
    'enough',
    'https://t.co/ojowdrc58o',
    '@forbes',
    'https://t.co/k7zwtovdbz',
    'hiring',
    'a',
    'chief',
    'innovation',
    'officer',
    'is',
    'a',
   

In [44]:
# Se obtiene el listado de palabras 100 palabras más frecuentes para cada unos de los 10 últimos tweet de los 100 twitter
# seguidos por @UDD_cl
l_pf_ls=[]
for i in range(len(l_ls_fw)):
    l_pf_ls.append(palabras_frecuentes(l_ls_fw[i][1],100))
len(l_pf_ls), l_pf_ls 

(100,
 [[('a', 6),
   ('de', 5),
   ('rt', 4),
   ('la', 4),
   ('del', 3),
   ('@uddventures', 3),
   ('para', 3),
   ('not', 2),
   ('innovation', 2),
   ('is', 2),
   ('@forbes', 2),
   ('innovación', 2),
   ('en', 2),
   ('tu', 2),
   ('4', 1),
   ('things', 1),
   ('every', 1),
   ('business', 1),
   ('leader', 1),
   ('should', 1),
   ('know', 1),
   ('about', 1),
   ('artificial', 1),
   ('intelligence', 1),
   ('and', 1),
   ('automation', 1),
   ('https://t.co/dykkxxkrqy', 1),
   ('@eduardobitran', 1),
   ('recomiendo', 1),
   ('esta', 1),
   ('entrevista', 1),
   ('acemoglu', 1),
   ('economista', 1),
   ('mit.', 1),
   ('autor', 1),
   ('libro', 1),
   ('porque', 1),
   ('fracasan', 1),
   ('los', 1),
   ('países', 1),
   ('favor', 1),
   ('nueva', 1),
   ('cons', 1),
   ('big', 1),
   ('corporations', 1),
   ('are', 1),
   ('rushing', 1),
   ('into', 1),
   ('venture', 1),
   ('capital.', 1),
   ('that', 1),
   ('may', 1),
   ('be', 1),
   ('good', 1),
   ('thing.', 1),
   

In [45]:
# Es mostrado el listado de 100 id de twitter seguidos por @UDD_cl junto con las 100 palabras más frecuentes de sus últimos 
# 10 tweet.
for i in range(len(l_seguidos)):
    print(l_seguidos[i],l_pf_ls[i])

13592012 [('a', 6), ('de', 5), ('rt', 4), ('la', 4), ('del', 3), ('@uddventures', 3), ('para', 3), ('not', 2), ('innovation', 2), ('is', 2), ('@forbes', 2), ('innovación', 2), ('en', 2), ('tu', 2), ('4', 1), ('things', 1), ('every', 1), ('business', 1), ('leader', 1), ('should', 1), ('know', 1), ('about', 1), ('artificial', 1), ('intelligence', 1), ('and', 1), ('automation', 1), ('https://t.co/dykkxxkrqy', 1), ('@eduardobitran', 1), ('recomiendo', 1), ('esta', 1), ('entrevista', 1), ('acemoglu', 1), ('economista', 1), ('mit.', 1), ('autor', 1), ('libro', 1), ('porque', 1), ('fracasan', 1), ('los', 1), ('países', 1), ('favor', 1), ('nueva', 1), ('cons', 1), ('big', 1), ('corporations', 1), ('are', 1), ('rushing', 1), ('into', 1), ('venture', 1), ('capital.', 1), ('that', 1), ('may', 1), ('be', 1), ('good', 1), ('thing.', 1), ('https://t.co/dffgpgihe6', 1), ('pathways', 1), ('to', 1), ('growth', 1), ('essential', 1), ('but', 1), ('enough', 1), ('https://t.co/ojowdrc58o', 1), ('https://t.

204441838 [('rt', 10), ('el', 8), ('de', 8), ('que', 7), ('y', 7), ('a', 5), ('en', 4), ('@sebastianpinera', 3), ('para', 3), ('los', 3), ('todos', 2), ('es', 2), ('por', 2), ('pero', 2), ('chile', 2), ('ha', 2), ('@ignaciobriones_', 2), ('poco', 2), ('la', 2), ('un', 2), ('más', 2), ('seremi', 2), ('@presidencia_cl', 1), ('ahora', 1), ('–', 1), ('pdte', 1), ('llega', 1), ('#uruguay', 1), ('participar', 1), ('transmisión', 1), ('mando', 1), ('asumirá', 1), ('como', 1), ('nuevo', 1), ('pre', 1), ('@mjose_zaldivar', 1), ('hola', 1), ('creo', 1), ('fundamental', 1), ('trabajadores', 1), ('conozcan', 1), ('sus', 1), ('derechos', 1), ('laborales.', 1), ('eso', 1), ('hoy', 1), ('quiero', 1), ('e', 1), ('coronavirus', 1), ('comenzó', 1), ('china', 1), ('se', 1), ('expande', 1), ('rápidamente', 1), ('mundo.', 1), ('tomado', 1), ('todas', 1), ('las', 1), ('medidas', 1), ('preventiva', 1), ('buenas', 1), ('tardes.', 1), ('hace', 1), ('lanzamos', 1), ('comisión', 1), ('#mejorgastopúblico.', 1), (

44136211 [('de', 11), ('rt', 9), ('y', 7), ('en', 6), ('@min_ciencia', 5), ('la', 5), ('a', 4), ('las', 4), ('más', 4), ('el', 4), ('una', 3), ('un', 3), ('sobre', 3), ('del', 3), ('@acouvecorrea', 3), ('con', 3), ('científicas', 2), ('historia', 2), ('al', 2), ('que', 2), ('jóvenes', 2), ('hoy', 2), ('@_somich', 2), ('para', 2), ('niñas', 2), ('por', 2), ('🌠hoy', 1), ('los', 1), ('101', 1), ('años', 1), ('murió', 1), ('katherine', 1), ('johnson', 1), ('importantes', 1), ('exploración', 1), ('esp', 1), ('@almaobs_esp', 1), ('📡junto', 1), ('vla', 1), ('@thenrao', 1), ('generamos', 1), ('300', 1), ('imágenes', 1), ('discos', 1), ('protoplanetarios', 1), ('rodean', 1), ('estrellas', 1), ('nubes', 1), ('diminuta', 1), ('belleza', 1), ('picaflor', 1), ('te', 1), ('invitamos', 1), ('conocer', 1), ('poco', 1), ('aves', 1), ('pequeñas', 1), ('mund', 1), ('https://t.co/zhqx3gdequ', 1), ('nos', 1), ('reunimos', 1), ('equipo', 1), ('@ministeriosalud', 1), ('investigadores', 1), ('e', 1), ('invest

396796970 [('de', 10), ('a', 9), ('para', 6), ('y', 6), ('en', 5), ('la', 4), ('🔵', 3), ('sus', 3), ('quieres', 2), ('ser', 2), ('voluntario/a', 2), ('ayudar', 2), ('personas', 2), ('resolver', 2), ('dudas', 2), ('migración', 2), ('refugio', 2), ('📲', 2), ('al', 2), ('con', 2), ('nuestro', 2), ('director', 2), ('incidencia', 2), ('estudios', 2), ('juan', 2), ('pablo', 2), ('ramaciotti', 2), ('@jpramaciotti', 2), ('las', 2), ('como', 2), ('#arica', 1), ('súmate', 1), ('https://t.co/pdqorf8v2m', 1), ('#antofagasta', 1), ('s', 1), ('https://t.co/kh0dnfk95e', 1), ('rt', 1), ('@redjesuitaconm1', 1), ('#chile', 1), ('#empleomigrantes', 1), ('gran', 1), ('iniciativa', 1), ('@sjmchile', 1), ('te', 1), ('encuentras', 1), ('búsqueda', 1), ('activa', 1), ('trabajo', 1), ('inscríbete', 1), ('programa', 1), ('laboral', 1), ('del', 1), ('sjm', 1), ('ingresando', 1), ('siguiente', 1), ('lin', 1), ('https://t.co/ep0tlahkii', 1), ('leer', 1), ('🧐', 1), ('atención', 1), ('@latercera', 1), ('tener', 1), 

16392202 [('en', 11), ('de', 8), ('la', 5), ('educación', 3), ('2020', 3), ('con', 3), ('más', 3), ('e', 3), ('qué', 3), ('o', 2), ('para', 2), ('rt', 2), ('realidad', 2), ('es', 2), ('por', 2), ('quieres', 2), ('sobre', 2), ('el', 2), ('y', 2), ('acompañamos', 2), ('del', 2), ('los', 2), ('📚', 2), ('⚠️', 1), ('te', 1), ('gustaría', 1), ('ser', 1), ('parte', 1), ('👉🏼buscamos', 1), ('periodista', 1), ('#valdivia', 1), ('tres', 1), ('años', 1), ('experiencia', 1), ('https://t.co/ryws4t8eek', 1), ('creemos', 1), ('que', 1), ('estamos', 1), ('viviendo', 1), ('una', 1), ('oportunidad', 1), ('histórica', 1), ('avanzar', 1), ('procesos', 1), ('democratizantes', 1), ('https://t.co/ejixkoksug', 1), ('@bpdigital_cl', 1), ('15', 1), ('cuentos', 1), ('ambientados', 1), ('un', 1), ('mundo', 1), ('voraz', 1), ('impersonal', 1), ('donde', 1), ('inmediata', 1), ('se', 1), ('confunde', 1), ('virtual.', 1), ('cita', 1), ('@mipantalonnegro', 1), ('esa', 1), ('muchas', 1), ('escuelas', 1), ('chile', 1), (

...

1702612722 [('de', 13), ('la', 8), ('del', 7), ('investigadora', 4), ('por', 4), ('el', 3), ('en', 2), ('gran', 2), ('y', 2), ('es', 2), ('estudio', 2), ('un', 2), ('febrero', 2), ('pero', 2), ('conocida', 2), ('que', 2), ('amalia', 2), ('como', 2), ('fleming', 2), ('dra.', 2), ('hoy', 1), ('@revistadomingo', 1), ('@elmercurio_cl', 1), ('bióloga', 1), ('doctora', 1), ('microbiología', 1), ('antofagastina', 1), ('cristina', 1), ('dorador', 1), ('https://t.co/im2howdabc', 1), ('ella', 1), ('nicole', 1), ('w.', 1), ('xu', 1), ('u.', 1), ('@stanford', 1), ('coautora', 1), ('control', 1), ('robótico', 1), ('natación', 1), ('d', 1), ('https://t.co/aypyrz6iz4', 1), ('27', 1), ('1869', 1), ('nació', 1), ('alice', 1), ('hamilton', 1), ('médico', 1), ('e', 1), ('estadounidense', 1), ('su', 1), ('trabajo', 1), ('https://t.co/e6fu7kfoyb', 1), ('proyecto', 1), ('liderado', 1), ('judith', 1), ('garcía', 1), ('aymerich', 1), ('@judithgarciaaym', 1), ('@isglobalorg', 1), ('demuestra', 1), ('aumento', 

15490422 [('de', 10), ('en', 6), ('el', 6), ('a', 5), ('los', 5), ('del', 5), ('y', 4), ('un', 4), ('que', 4), ('the', 4), ('la', 3), ('nuestros', 3), ('estudiantes', 3), ('todos', 3), ('campus', 3), ('semestre', 3), ('@tecdemonterrey', 3), ('líderes', 2), ('mi', 2), ('por', 2), ('me', 2), ('to', 2), ('cierre', 2), ('in', 2), ('marco', 1), ('inauguración', 1), ('edición', 1), ('número', 1), ('50', 1), ('nuestro', 1), ('congreso', 1), ('investigación', 1), ('desarrollo', 1), ('les', 1), ('compa', 1), ('https://t.co/wf2vdfhqpk', 1), ('unas', 1), ('horas', 1), ('recibiremos', 1), ('@tecdemonterrey.', 1), ('inicia', 1), ('nuevo', 1), ('https://t.co/kp0lndujxa', 1), ('@bancomundial', 1), ('publicó', 1), ('caso', 1), ('rompiendo', 1), ('paradigmas', 1), ('para', 1), ('desarrollar', 1), ('siglo', 1), ('xxi', 1), ('narra', 1), ('pr', 1), ('https://t.co/o0zte06pbn', 1), ('paso', 1), ('distintas', 1), ('responsabilidades', 1), ('tuve', 1), ('oportunidad', 1), ('reportarle', 1), ('https://t.co/ef

346338347 [('de', 19), ('rt', 7), ('y', 6), ('en', 4), ('proyectos', 4), ('@energiavalpo', 3), ('agenciase', 3), ('para', 3), ('implementación', 3), ('#cogeneración', 3), ('se', 3), ('#renca', 3), ('#navidad', 3), ('del', 3), ('concurso', 3), ('a', 2), ('@agenciadese', 2), ('inicia', 2), ('hospitales', 2), ('#puertonatales', 2), ('#porvenir', 2), ('⚡', 2), ('la', 2), ('iniciativas', 2), ('#peñalolén', 2), ('adjudican', 2), ('tercer', 2), ('inversión', 2), ('energética', 2), ('local', 2), ('👉🏼profesionales', 1), ('junto', 1), ('@seremimineraval', 1), ('#leopoldovalenzuela', 1), ('visitan', 1), ('pirquineros👷🏻\u200d♂️', 1), ('chincolco', 1), ('m', 1), ('adjudica', 1), ('licitación', 1), ('acondicionamiento', 1), ('térmico', 1), ('liceo', 1), ('enrique', 1), ('molina', 1), ('garmendia', 1), ('concepción', 1), ('inici', 1), ('https://t.co/xjd0xfnuhw', 1), ('https://t.co/y', 1), ('https://t.co/9g4bcnu1pi', 1), ('seremi', 1), ('@mfpinochet', 1), ('presenta', 1), ('desafío', 1), ('#innovafosi

113691620 [('de', 11), ('que', 10), ('con', 7), ('en', 6), ('y', 5), ('opinión', 4), ('la', 4), ('a', 4), ('nuestros', 4), ('los', 4), ('el', 3), ('una', 3), ('un', 3), ('estarás', 3), ('aportando', 3), ('para', 3), ('@jfyksj', 2), ('#díamundialdelamujer', 2), ('esperemos', 2), ('creatividad', 2), ('no', 2), ('agresiva', 2), ('ellas', 2), ('sea', 2), ('forma', 2), ('poten', 2), ('ad', 2), ('portas', 2), ('#marzo', 2), ('vaticinan', 2), ('incendiario', 2), ('términos', 2), ('violencia', 2), ('espero', 2), ('podamos', 2), ('debatir', 2), ('co', 2), ('niños', 2), ('niñas', 2), ('productos', 2), ('#vueltaaclases', 2), ('te', 2), ('contamos', 2), ('del', 2), ('https://t.co/iiktapcozm', 1), ('https://t.co/vwouyo7rkw', 1), ('vuelve', 1), ('#clases', 1), ('accesorios', 1), ('jardines', 1), ('infantiles', 1), ('m', 1), ('https://t.co/fgrm0cythm', 1), ('personas', 1), ('#situacióndecalle', 1), ('reciben', 1), ('kit', 1), ('#verano', 1), ('bloqueadores', 1), ('solares', 1), ('alcohol', 1), ('gel'

284074479 [('de', 18), ('la', 10), ('el', 8), ('rt', 6), ('a', 6), ('del', 5), ('periodistas', 5), ('y', 4), ('que', 4), ('colegio', 4), ('en', 3), ('día', 3), ('al', 3), ('los', 2), ('prensa.', 2), ('consejo', 2), ('metropolitano', 2), ('@chileperiodista', 2), ('por', 2), ('qué', 2), ('celebrar', 2), ('es', 2), ('chile', 2), ('para', 2), ('@lorepenjean', 1), ('the', 1), ('clinic', 1), ('manifiesta', 1), ('su', 1), ('total', 1), ('rechazo', 1), ('cobardes', 1), ('ataques', 1), ('contra', 1), ('ejercicio', 1), ('libertad', 1), ('“grupo', 1), ('mani', 1), ('@periodistas_rm', 1), ('desde', 1), ('regional', 1), ('condenamos', 1), ('categóricamente', 1), ('agresión', 1), ('robo', 1), ('sufrieron', 1), ('nuestro', 1), ('respaldo', 1), ('colegas', 1), ('@thecliniccl', 1), ('agredidos', 1), ('mientras', 1), ('cubrían', 1), ('manifestación', 1), ('favor', 1), ('#rechazo.', 1), ('repudiamo', 1), ('https://t.co/azqrowmlmg', 1), ('#diadelaprensa', 1), ('seguimos', 1), ('denunciado', 1), ('violacio

In [46]:
# Es presentada una segunda interpretación del ejercicio 2.d, en donde se realiza un análisis de frecuencia de palabras,
# para todos los tweet a la vez, sin hacer una separación por id de twitter.
# Se guardan todos los tweet en un string para su posterior tratamiento.
s_all_tweet=''
for tweet in l_seguidos_tweet:
    s_all_tweet += tweets_to_string(tweet)
# Es mostrado s_all_tweet
s_all_tweet

'4 Things Every Business Leader Should Know About Artificial Intelligence and Automation https://t.co/DYkkXXkRqY RT @EduardoBitran: Recomiendo esta entrevista a Acemoglu, Economista MIT. Autor del libro Porque fracasan los países? A favor de nueva Cons… Big Corporations Are Rushing Into Venture Capital. That May Not Be a Good Thing. https://t.co/dffGpGIhE6 Pathways to Growth: Innovation is essential, but not enough https://t.co/oJoWDrC58o @Forbes https://t.co/k7ZWTOVdbZ Hiring A Chief Innovation Officer Is A Bad Idea via @forbes https://t.co/i8wTVDIS5b RT @UDDVentures: Nuestro partner InterSystems está lanzando una funcionalidad global para su sistema unificado de información de salud Trak… RT @UDDVentures: "Innovación en la sociedad" Columna de Angel Morales sobre la importancia del aporte de la innovación para generar nuevos… RT @UDDVentures: Toma la decisión correcta para lograr hacer crecer tu negocio. Asegura tu cupo en el programa UDD Ventures Transforma. Esc… @RobertoKPax @Fredd

In [47]:
# Se toquenizan todas las palabras de todos los tweet rescatados.
ln_s_all_tweet_t, l_s_all_tweet_t = tokenizador(ToktokTokenizer,s_all_tweet)
# Se muestra el listado de palabras toquenizas.
ln_s_all_tweet_t, l_s_all_tweet_t 

(20470,
 ['4',
  'things',
  'every',
  'business',
  'leader',
  'should',
  'know',
  'about',
  'artificial',
  'intelligence',
  'and',
  'automation',
  'https://t.co/dykkxxkrqy',
  'rt',
  '@eduardobitran',
  ':',
  'recomiendo',
  'esta',
  'entrevista',
  'a',
  'acemoglu',
  ',',
  'economista',
  'mit.',
  'autor',
  'del',
  'libro',
  'porque',
  'fracasan',
  'los',
  'países',
  '?',
  'a',
  'favor',
  'de',
  'nueva',
  'cons',
  '…',
  'big',
  'corporations',
  'are',
  'rushing',
  'into',
  'venture',
  'capital.',
  'that',
  'may',
  'not',
  'be',
  'a',
  'good',
  'thing.',
  'https://t.co/dffgpgihe6',
  'pathways',
  'to',
  'growth',
  ':',
  'innovation',
  'is',
  'essential',
  ',',
  'but',
  'not',
  'enough',
  'https://t.co/ojowdrc58o',
  '@forbes',
  'https://t.co/k7zwtovdbz',
  'hiring',
  'a',
  'chief',
  'innovation',
  'officer',
  'is',
  'a',
  'bad',
  'idea',
  'via',
  '@forbes',
  'https://t.co/i8wtvdis5b',
  'rt',
  '@uddventures',
  ':',


In [48]:
# Se filtran las palabras stop_words del listado de palabras toquenizadas

ln_s_all_tweet_fw, l_s_all_tweet_fw = filtra_palabras(l_s_all_tweet_t ,l_sw)

# Se muestra el listado de palabras ya filtrado 
ln_s_all_tweet_fw, l_s_all_tweet_fw 

(17883,
 ['4',
  'things',
  'every',
  'business',
  'leader',
  'should',
  'know',
  'about',
  'artificial',
  'intelligence',
  'and',
  'automation',
  'https://t.co/dykkxxkrqy',
  'rt',
  '@eduardobitran',
  'recomiendo',
  'esta',
  'entrevista',
  'a',
  'acemoglu',
  'economista',
  'mit.',
  'autor',
  'del',
  'libro',
  'porque',
  'fracasan',
  'los',
  'países',
  'a',
  'favor',
  'de',
  'nueva',
  'cons',
  'big',
  'corporations',
  'are',
  'rushing',
  'into',
  'venture',
  'capital.',
  'that',
  'may',
  'not',
  'be',
  'a',
  'good',
  'thing.',
  'https://t.co/dffgpgihe6',
  'pathways',
  'to',
  'growth',
  'innovation',
  'is',
  'essential',
  'but',
  'not',
  'enough',
  'https://t.co/ojowdrc58o',
  '@forbes',
  'https://t.co/k7zwtovdbz',
  'hiring',
  'a',
  'chief',
  'innovation',
  'officer',
  'is',
  'a',
  'bad',
  'idea',
  'via',
  '@forbes',
  'https://t.co/i8wtvdis5b',
  'rt',
  '@uddventures',
  'nuestro',
  'partner',
  'intersystems',
  'es

In [49]:
# Son obtenidas las 100 palabras más frecuentes de todos los tweet.
l_pf_s_all_tweet = palabras_frecuentes(l_s_all_tweet_fw,100)
l_pf_s_all_tweet

[('de', 974),
 ('la', 537),
 ('el', 460),
 ('en', 451),
 ('y', 348),
 ('a', 344),
 ('rt', 343),
 ('que', 244),
 ('del', 190),
 ('para', 169),
 ('los', 164),
 ('con', 150),
 ('un', 142),
 ('las', 131),
 ('por', 125),
 ('es', 120),
 ('se', 115),
 ('una', 98),
 ('al', 89),
 ('no', 86),
 ('más', 62),
 ('hoy', 61),
 ('su', 50),
 ('the', 50),
 ('te', 49),
 ('sobre', 48),
 ('&#124;', 48),
 ('lo', 47),
 ('chile', 46),
 ('esta', 37),
 ('of', 37),
 ('como', 36),
 ('to', 35),
 ('este', 35),
 ('nuestro', 33),
 ('si', 33),
 ('in', 30),
 ('años', 29),
 ('sus', 29),
 ('nos', 29),
 ('tu', 28),
 ('día', 28),
 ('nuestra', 28),
 ('trabajo', 27),
 ('junto', 26),
 ('todos', 26),
 ('país', 25),
 ('desde', 25),
 ('2020', 25),
 ('me', 24),
 ('ha', 24),
 ('and', 23),
 ('está', 23),
 ('pero', 23),
 ('e', 23),
 ('ya', 23),
 ('o', 22),
 ('ahora', 22),
 ('ser', 22),
 ('qué', 21),
 ('son', 21),
 ('entrevista', 20),
 ('cómo', 20),
 ('on', 20),
 ('muy', 19),
 ('social', 19),
 ('educación', 19),
 ('d', 19),
 ('nuestro